In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:////code/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

In [ ]:
from langchain_community.utilities import SQLDatabase
from IPython.display import display
import pandas as pd
from sqlalchemy import create_engine

# Initialize the database connection
db_uri = "sqlite:////code/Chinook.db"
db = SQLDatabase.from_uri(db_uri)

# Display the database dialect and usable table names
display(db.dialect)
display(db.get_usable_table_names())

# Create a SQLAlchemy engine using the database URI
engine = create_engine(db_uri)

# Define your SQL query
query = "SELECT * FROM Artist LIMIT 10;"

# Execute the query and store the results in a DataFrame
df = pd.read_sql_query(query, engine)

# Display the DataFrame
display(df)


In [ ]:
from langchain_openai import AzureChatOpenAI
from os import environ
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    temperature=0.2,
    api_version="2023-06-01-preview",
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "how many employeed are there?"})
response

In [ ]:
from langchain_core.messages import HumanMessage
# Define the chain with the additional step to clean the response
def clean_sql_query(response):
    clean_response = llm.invoke([HumanMessage(content=f"Here is a string that contains the sql query {response}, I want you to extract the sql query from this string and return only the query and nothing else, so I can pass it directly in my code, don't prefix it with ```sql")])
    return clean_response.content

In [ ]:
clean_sql_query(response)

In [ ]:
chain.get_prompts()[0].pretty_print()

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | clean_sql_query | execute_query
chain.invoke({"question": "How many employees are there"})

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query | clean_sql_query)
    .assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "Who are AC/DC's biggest fans?"})

In [ ]:
write_query_chain = create_sql_query_chain(llm, db)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins
- Don't output anything for markup or formatting like ```sql```

If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": write_query_chain} | validation_chain

In [ ]:
query = full_chain.invoke(
    {
        "question": "How many employees are there"
    }
)
print(query)

In [ ]:
chain = (
    RunnablePassthrough.assign(query=full_chain)
    .assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many employees are there"})

In [ ]:
system = """You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
don't use LIMIT unless the user asks for a specific number of examples


Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>

Don't output anything for markup or formatting like ```sql```

"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer
prompt.pretty_print()

In [ ]:
query = chain.invoke(
    {
        "question": "shows the invoices associated with each sales agent. The result should include the Sales Agents full name"
    }
)
print(query)

In [ ]:
chain = (
    RunnablePassthrough.assign(query=full_chain)
    .assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "shows the invoices associated with each sales agent. The result should include the Sales Agents full name"})

In [ ]:
from langchain.chains.sql_database.prompt import SQL_PROMPTS

list(SQL_PROMPTS)

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
chain.get_prompts()[0].pretty_print()

In [ ]:
context = db.get_context()
print(list(context))
print(context["table_info"])

In [ ]:
prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])
print(prompt_with_context.pretty_repr()[:1500])